# Swift DeepSky over Stripe82 cleaning

We have two tables -- count rates and nufnu fluxes -- containing the combined results of the SDS pipeline over all Swift observations within the Stripe82 region.

Most probably these tables contain duplicated entries, which we want to remove.
The catalog we want is to be made by unique sources, usually called *primary* sources, that score better is a given property.

The criterion we will use to select filter the sources is their signal-to-noise ratio in the `full` band: $0.3-10 keV$; *i.e*, all detected events.
The object that scores better among its duplicates guarantees its place in the final catalog.

The cross-matching and filtering process is done by `xmatch`, we give it the coordinates and selection criterion columns and it outputs the primary catalog.

## Filter catalogs

Filenames:
* `table_countrates_detections_stripe82.csv`
* `table_flux_detections_stripe82.csv`

### the photon flux catalog

In [1]:
!head -n2 table_countrates_detections_stripe82.csv

RA;DEC;countrates_0.3-10keV(ph.s-1);countrates_error_0.3-10keV(ph.s-1);exposure_time(s);countrates_0.3-1keV(ph.s-1);countrates_error_0.3-1keV(ph.s-1);upper_limit_0.3-1keV(ph.s-1);countrates_1-2keV(ph.s-1);countrates_error_1-2keV(ph.s-1);upper_limit_1-2keV(ph.s-1);countrates_2-10keV(ph.s-1);countrates_error_2-10keV(ph.s-1);upper_limit_2-10keV(ph.s-1)
00:56:24.480;-01:16:38.317;5.645E-03;1.500E-03;4572.8;8.684E-04;6.000E-04;-9.990E+02;3.040E-03;1.125E-03;-9.990E+02;1.737E-03;8.531E-04;-9.990E+02


In [34]:
import pandas

dfc = pandas.read_csv('table_countrates_detections_stripe82.csv', sep=';')
dfc.head()

,RA,DEC,countrates_0.3-10keV(ph.s-1),countrates_error_0.3-10keV(ph.s-1),exposure_time(s),countrates_0.3-1keV(ph.s-1),countrates_error_0.3-1keV(ph.s-1),upper_limit_0.3-1keV(ph.s-1),countrates_1-2keV(ph.s-1),countrates_error_1-2keV(ph.s-1),upper_limit_1-2keV(ph.s-1),countrates_2-10keV(ph.s-1),countrates_error_2-10keV(ph.s-1),upper_limit_2-10keV(ph.s-1)
0,00:56:24.480,-01:16:38.317,0.005645,0.0015,4572.8,0.000868,0.000600,-999.0,0.003040,0.001125,-999.0,0.001737,0.000853,-999.0
1,00:56:19.136,-01:14:58.198,0.004449,0.0014,4684.2,0.000494,0.000485,-999.0,0.001977,0.000959,-999.0,0.001977,0.000959,-999.0
2,00:56:23.004,-01:13:39.516,0.003901,0.0014,4649.6,0.000867,0.000679,-999.0,0.001301,0.000840,-999.0,0.001734,0.000969,-999.0
3,00:56:16.418,-01:14:05.418,0.005795,0.0016,4642.0,0.000446,0.000460,-999.0,0.002674,0.001100,-999.0,0.002674,0.001100,-999.0
4,00:56:16.922,-01:13:16.401,0.005224,0.0015,4587.3,0.002177,0.000937,-999.0,0.001306,0.000741,-999.0,0.001741,0.000853,-999.0


*Every catalog has to have an ID column; let's give one to it.*

In [35]:
dfc.index.name = 'OBJID'
dfc = dfc.reset_index()
dfc.head()

,OBJID,RA,DEC,countrates_0.3-10keV(ph.s-1),countrates_error_0.3-10keV(ph.s-1),exposure_time(s),countrates_0.3-1keV(ph.s-1),countrates_error_0.3-1keV(ph.s-1),upper_limit_0.3-1keV(ph.s-1),countrates_1-2keV(ph.s-1),countrates_error_1-2keV(ph.s-1),upper_limit_1-2keV(ph.s-1),countrates_2-10keV(ph.s-1),countrates_error_2-10keV(ph.s-1),upper_limit_2-10keV(ph.s-1)
0,0,00:56:24.480,-01:16:38.317,0.005645,0.0015,4572.8,0.000868,0.000600,-999.0,0.003040,0.001125,-999.0,0.001737,0.000853,-999.0
1,1,00:56:19.136,-01:14:58.198,0.004449,0.0014,4684.2,0.000494,0.000485,-999.0,0.001977,0.000959,-999.0,0.001977,0.000959,-999.0
2,2,00:56:23.004,-01:13:39.516,0.003901,0.0014,4649.6,0.000867,0.000679,-999.0,0.001301,0.000840,-999.0,0.001734,0.000969,-999.0
3,3,00:56:16.418,-01:14:05.418,0.005795,0.0016,4642.0,0.000446,0.000460,-999.0,0.002674,0.001100,-999.0,0.002674,0.001100,-999.0
4,4,00:56:16.922,-01:13:16.401,0.005224,0.0015,4587.3,0.002177,0.000937,-999.0,0.001306,0.000741,-999.0,0.001741,0.000853,-999.0


*Let's transform RA and DEC in degrees*

In [36]:
from astropy.coordinates import SkyCoord
from astropy import units
coords = SkyCoord(dfc['RA'], dfc['DEC'], unit=(units.hourangle,units.degree))
ra = coords.ra
ra.wrap_angle = 180 * units.deg
dec = coords.dec
dfc['RA'] = ra.deg
dfc['DEC'] = dec.deg

In [37]:
dfc.head()

,OBJID,RA,DEC,countrates_0.3-10keV(ph.s-1),countrates_error_0.3-10keV(ph.s-1),exposure_time(s),countrates_0.3-1keV(ph.s-1),countrates_error_0.3-1keV(ph.s-1),upper_limit_0.3-1keV(ph.s-1),countrates_1-2keV(ph.s-1),countrates_error_1-2keV(ph.s-1),upper_limit_1-2keV(ph.s-1),countrates_2-10keV(ph.s-1),countrates_error_2-10keV(ph.s-1),upper_limit_2-10keV(ph.s-1)
0,0,14.102000,-1.277310,0.005645,0.0015,4572.8,0.000868,0.000600,-999.0,0.003040,0.001125,-999.0,0.001737,0.000853,-999.0
1,1,14.079733,-1.249499,0.004449,0.0014,4684.2,0.000494,0.000485,-999.0,0.001977,0.000959,-999.0,0.001977,0.000959,-999.0
2,2,14.095850,-1.227643,0.003901,0.0014,4649.6,0.000867,0.000679,-999.0,0.001301,0.000840,-999.0,0.001734,0.000969,-999.0
3,3,14.068408,-1.234838,0.005795,0.0016,4642.0,0.000446,0.000460,-999.0,0.002674,0.001100,-999.0,0.002674,0.001100,-999.0
4,4,14.070508,-1.221223,0.005224,0.0015,4587.3,0.002177,0.000937,-999.0,0.001306,0.000741,-999.0,0.001741,0.000853,-999.0


In [38]:
dfc['snr'] = dfc['countrates_0.3-10keV(ph.s-1)'] / dfc['countrates_error_0.3-10keV(ph.s-1)']

In [39]:
dfc.describe()

,OBJID,RA,DEC,countrates_0.3-10keV(ph.s-1),countrates_error_0.3-10keV(ph.s-1),exposure_time(s),countrates_0.3-1keV(ph.s-1),countrates_error_0.3-1keV(ph.s-1),upper_limit_0.3-1keV(ph.s-1),countrates_1-2keV(ph.s-1),countrates_error_1-2keV(ph.s-1),upper_limit_1-2keV(ph.s-1),countrates_2-10keV(ph.s-1),countrates_error_2-10keV(ph.s-1),upper_limit_2-10keV(ph.s-1),snr
count,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000,6905.000000
mean,3452.000000,18.919874,-0.199491,0.010191,0.000634,46500.832324,0.004180,-0.044353,-942.006136,0.003567,-0.066411,-948.227229,0.002757,-0.102818,-924.355174,6.560666
std,1993.446137,32.071227,0.786714,0.099481,0.001161,55869.816221,0.040574,0.258776,234.133861,0.034654,0.528543,221.976278,0.027086,0.613516,264.822428,12.399557
min,0.000000,-58.027692,-1.639452,0.000119,0.000036,387.500000,0.000000,-5.195000,-2530.000000,0.000000,-13.990000,-2530.000000,0.000000,-13.990000,-2530.000000,2.040541
25%,1726.000000,-3.214592,-0.862277,0.000666,0.000140,9247.100000,0.000169,0.000060,-999.000000,0.000200,0.000066,-999.000000,0.000189,0.000063,-999.000000,3.288571
50%,3452.000000,29.718933,-0.274189,0.001447,0.000330,20961.300000,0.000433,0.000150,-999.000000,0.000467,0.000160,-999.000000,0.000412,0.000139,-999.000000,4.284375
75%,5178.000000,43.782867,0.404549,0.003039,0.000680,67671.400000,0.001099,0.000364,-999.000000,0.001115,0.000370,-999.000000,0.000935,0.000317,-999.000000,6.215000
max,6904.000000,60.102000,1.606157,2.716000,0.023000,295994.000000,1.091000,0.015680,0.082480,0.911400,0.014640,0.117300,0.944600,0.008259,0.136400,290.219780


In [40]:
cols = dict(ra='RA', dec='DEC', id='OBJID')

In [41]:
from astropy.coordinates import Angle
rad = Angle(5,'arcsec')

In [42]:
from xmatch import xmatch

%time xcatc = xmatch(dfc, dfc, cols, cols, radius=rad, snr_column='snr')

/home/chbrandt/.conda/envs/xmatch/lib/python3.6/site-packages/pandas/core/computation/expressions.py:179: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


CPU times: user 28.4 s, sys: 20.1 ms, total: 28.4 s
Wall time: 28.4 s


/home/chbrandt/.conda/envs/xmatch/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/chbrandt/.conda/envs/xmatch/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [43]:
xcatc.describe()

A                                      B               \
                RA          DEC        OBJID           RA          DEC   
count  2764.000000  2764.000000  2764.000000  2764.000000  2764.000000   
mean     15.019202    -0.106151  2275.367945    15.019201    -0.106153   
std      31.970747     0.787671  1726.748884    31.970745     0.787670   
min     -58.027692    -1.639452     0.000000   -58.027692    -1.639452   
25%      -7.908507    -0.791660   837.750000    -7.908507    -0.791660   
50%      21.870021    -0.159245  1790.500000    21.870021    -0.159245   
75%      40.490854     0.516338  3476.500000    40.490854     0.516338   
max      60.102000     1.606157  6736.000000    60.102000     1.606157   

                             AB  
             OBJID          snr  
count  2764.000000  2764.000000  
mean   2597.801013     6.109273  
std    1798.401101    10.587388  
min       0.000000     2.040541  
25%    1135.750000     3.102268  
50%    2297.500000     4.045500  
75%    3978.250000     5.936814  
max    6877.000000   290.219780

In [12]:
# xcatc.to_csv('xmatch_table_countrates_detections_stripe82_unique.csv')

### The energy flux catalog

In [56]:
detections_filename = 'table_flux_detections_stripe82.csv'
flux_column = 'nufnu_3keV(erg.s-1.cm-2)'
flux_error_column = 'nufnu_error_3keV(erg.s-1.cm-2)'

import pandas
df = pandas.read_csv(detections_filename, sep=';')

df.index.name = 'OBJID'
df = df.reset_index()
df.head()

from astropy.coordinates import SkyCoord
from astropy import units
coords = SkyCoord(df['RA'], df['DEC'], unit=(units.hourangle,units.degree))
ra = coords.ra
ra.wrap_angle = 180 * units.deg
dec = coords.dec
df['RA'] = ra.deg
df['DEC'] = dec.deg

df['snr'] = df[flux_column] / df[flux_error_column]
from xmatch import xmatch
cols = dict(ra='RA', dec='DEC', id='OBJID')

from astropy.coordinates import Angle
rad = Angle(5,'arcsec')

xcat = xmatch(df, df, cols, cols, radius=rad, snr_column='snr')

/home/chbrandt/.conda/envs/xmatch/lib/python3.6/site-packages/pandas/core/computation/expressions.py:179: UserWarning: evaluating in Python space because the '*' operator is not supported by numexpr for the bool dtype, use '&' instead
  .format(op=op_str, alt_op=unsupported[op_str]))
/home/chbrandt/.conda/envs/xmatch/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/home/chbrandt/.conda/envs/xmatch/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [57]:
xcat.describe()

A                                      B               \
                RA          DEC        OBJID           RA          DEC   
count  2764.000000  2764.000000  2764.000000  2764.000000  2764.000000   
mean     15.019202    -0.106151  2275.367945    15.019201    -0.106153   
std      31.970747     0.787671  1726.748884    31.970745     0.787670   
min     -58.027692    -1.639452     0.000000   -58.027692    -1.639452   
25%      -7.908507    -0.791660   837.750000    -7.908507    -0.791660   
50%      21.870021    -0.159245  1790.500000    21.870021    -0.159245   
75%      40.490854     0.516338  3476.500000    40.490854     0.516338   
max      60.102000     1.606157  6736.000000    60.102000     1.606157   

                             AB  
             OBJID          snr  
count  2764.000000  2764.000000  
mean   2599.712735     6.109273  
std    1799.211485    10.587381  
min       0.000000     2.040551  
25%    1136.750000     3.102268  
50%    2298.500000     4.045499  
75%    3979.250000     5.936806  
max    6877.000000   290.218897

In [15]:
# xcat.to_csv('xmatch_table_flux_detections_stripe82_unique.csv')

## Checking the results

In [60]:
from bokeh.plotting import figure
from bokeh.io import output_notebook, show

output_notebook()

fig = figure()

from astropy.coordinates import Angle
r = Angle(5,'arcsec').degree

xa = df['RA']
ya = df['DEC']
fig.circle(xa, ya, fill_alpha=0.3, fill_color='red')

x = xcat[('A','RA')]
y = xcat[('A','DEC')]
fig.circle(x, y, fill_alpha=0.3, fill_color='blue')

show(fig)

Loading BokehJS ...

In [67]:
xcat[('A','OBJID')].duplicated().any()

False

In [68]:
xcat[('B','OBJID')].duplicated().any()

False

In [75]:
objid_a = xcat[('A','OBJID')].astype(int)
objid_b = xcat[('B','OBJID')].astype(int)
xcat.loc[~(objid_a == objid_b)]

A                          B                           AB  \
             RA       DEC OBJID         RA       DEC   OBJID        snr   
136   40.762221  0.774288   136  40.762333  0.774428   523.0  23.708321   
137   40.752762  0.699425   137  40.752567  0.700166   627.0   6.768757   
138   40.909400  0.742702   138  40.909562  0.742916   628.0  10.029046   
139   40.647475  0.767925   139  40.647033  0.767921   526.0   7.232762   
140   40.866904  0.931193   140  40.866554  0.931108   633.0   9.885719   
141   40.982917  0.715763   141  40.982563  0.715536   635.0  15.216649   
142   40.668442  0.956962   142  40.668492  0.957585   529.0  47.291620   
171   17.679000  0.336280   171  17.678825  0.337546  6626.0   4.552161   
179   17.851929  0.445452   179  17.852179  0.445501  6633.0   5.435833   
188   -0.983988 -0.398020   188  -0.983846 -0.398281  1570.0   9.682605   
189   -1.109917 -0.470031   189  -1.109938 -0.469951  1577.0   4.380355   
192   -0.979046 -0.579926   192  -0.979046 -0.579959  1580.0   6.999164   
229   45.902100 -1.032663   229  45.902100 -1.032663  2170.0   4.358115   
230   45.954592 -1.103661   230  45.954588 -1.103607  2171.0  18.719110   
231   45.922783 -1.015156   231  45.922783 -1.015156  2172.0   3.099998   
232   45.871796 -1.085104   232  45.871796 -1.085104  2173.0   6.284226   
234   45.858667 -1.050515   234  45.858667 -1.050515  2175.0   3.572143   
235   45.857554 -1.087446   235  45.857192 -1.087285  2176.0   3.955001   
236   46.002496 -1.114499   236  46.002475 -1.114418  2177.0   5.192765   
237   45.988775 -1.151901   237  45.988754 -1.151906  2178.0   7.755552   
238   45.827246 -1.063201   238  45.827246 -1.063201  2179.0   3.337143   
239   45.888075 -0.966263   239  45.888075 -0.966263  2180.0   3.698751   
240   45.821325 -1.064319   240  45.821325 -1.064319  2182.0   4.776472   
241   45.987183 -0.975139   241  45.987183 -0.975139  2181.0   3.440716   
243   45.978287 -0.954177   243  45.978375 -0.953967  2185.0   3.609285   
244   45.955029 -0.938779   244  45.955304 -0.938634  2186.0   6.278259   
245   45.917083 -1.192543   245  45.917083 -1.192543  2187.0   2.886665   
246   45.919321 -1.200373   246  45.919321 -1.200373  2188.0   2.684998   
247   46.074271 -1.095059   247  46.074304 -1.094828  2189.0   6.560864   
248   45.856779 -1.203958   248  45.856779 -1.203958  2190.0   3.638750   
...         ...       ...   ...        ...       ...     ...        ...   
2602  57.214354 -0.926323  2602  57.214354 -0.926323  4896.0   2.936114   
2610  57.266783 -1.002548  2610  57.266783 -1.002548  4904.0   3.134208   
2619  57.121117 -0.896086  2619  57.121117 -0.896086  3357.0   3.252384   
2641  57.055267 -1.004126  2641  57.055267 -1.004126  3380.0   6.211770   
2642  57.323825 -1.143947  2642  57.323825 -1.143947  4938.0  16.667328   
2744  30.445442  1.285622  2744  30.445050  1.285859  4223.0  20.418930   
2808  57.242650 -1.144839  2808  57.242650 -1.144839  2853.0   5.778957   
2812  57.333971 -1.124974  2812  57.333542 -1.124974  2859.0   6.133338   
2816  57.352375 -1.175393  2816  57.352375 -1.175393  2864.0  21.350035   
2931 -26.927467  0.540965  2931 -26.927467  0.540965  4705.0   6.304172   
2932 -26.992079  0.562283  2932 -26.992079  0.562283  4706.0   3.918741   
2935 -27.047158  0.535435  2935 -27.047158  0.535435  4709.0   4.686111   
2938 -26.824063  0.492290  2938 -26.823313  0.492617  4712.0   4.515781   
2942 -26.876867  0.683525  2942 -26.877129  0.683371  4714.0   9.906505   
2943 -26.757850  0.399065  2943 -26.757850  0.399065  4716.0  14.169228   
2944 -26.738925  0.503829  2944 -26.738796  0.503981  4715.0  12.608327   
2946 -26.713325  0.615138  2946 -26.713429  0.614866  4718.0  85.327357   
2951  29.739083  1.160135  2951  29.739083  1.160135  6501.0   2.570830   
2990  49.572692 -0.296278  2990  49.572688 -0.296622  3949.0  10.304518   
2991  49.687600 -0.313261  2991  49.687600 -0.313261  3952.0   8.076923   
2992  49.651550 -0.370554  2992  49.651

In [73]:
import numpy as np
np.array_equal(objid_a, objid_b)

False

To check the x-matching results we may use the distance between nearest-neighbours: neighbours should *not* be closer than the matching radius, $5 arcsec$.

In [76]:
from sklearn.neighbors import NearestNeighbors

X = xcat[[('B','RA'),('B','DEC')]].reset_index(drop=True)

In [77]:
nbrs = NearestNeighbors(n_neighbors=2, algorithm='kd_tree').fit(X)
nbrs

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=1, n_neighbors=2, p=2, radius=1.0)

In [78]:
distances, indices = nbrs.kneighbors(X)

In [79]:
import pandas
nn = pandas.DataFrame(dict(dist=distances[:,1]))
nn.describe()

,dist
count,2764.000000
mean,0.056498
std,0.078995
min,0.000782
25%,0.019657
50%,0.039002
75%,0.073537
max,1.792473


In [80]:
nn_pos = pandas.DataFrame(xcat[[('A','RA'),('A','DEC')]].iloc[indices[:,1]]).reset_index(drop=True)
nn_pos.columns = ('ra','dec')
nn_pos.describe()

,ra,dec
count,2764.000000,2764.000000
mean,15.020695,-0.106589
std,31.971398,0.784769
min,-58.027692,-1.596909
25%,-7.908507,-0.790958
50%,21.837500,-0.161073
75%,40.497263,0.517346
max,58.626412,1.532032


In [81]:
Xnn = pandas.concat((X,nn,nn_pos), axis=1)
Xnn

,"(B, RA)","(B, DEC)",dist,ra,dec
0,14.102000,-1.277310,0.035626,14.079733,-1.249499
1,14.079733,-1.249499,0.018526,14.068408,-1.234838
2,14.095850,-1.227643,0.026142,14.070508,-1.221223
3,14.068408,-1.234838,0.013777,14.070508,-1.221223
4,14.070508,-1.221223,0.013777,14.068408,-1.234838
5,14.055663,-1.248697,0.018829,14.068408,-1.234838
6,14.107788,-1.356168,0.079070,14.102000,-1.277310
7,14.012692,-1.239376,0.043970,14.055663,-1.248697
8,-3.830946,-0.202381,0.090550,-3.740417,-0.200416
9,-3.813942,-0.112661,0.091317,-3.830946,-0.202381


In [82]:
Xnn.sort_values('dist').head(20)

,"(B, RA)","(B, DEC)",dist,ra,dec
425,17.605092,-0.391549,0.000782,17.604379,-0.391872
1547,17.604379,-0.391872,0.000782,17.606217,-0.391604
445,58.255379,-0.050758,0.001132,58.255258,-0.051884
931,58.255258,-0.051884,0.001132,58.255354,-0.050074
456,58.452225,0.069827,0.001208,58.452225,0.069827
2088,58.452196,0.071034,0.001208,58.452625,0.069701
951,58.450958,0.071449,0.001305,58.452225,0.069827
454,58.237733,0.091080,0.001388,58.237029,0.089812
1412,58.236925,0.089952,0.001388,58.237733,0.091080
638,33.704187,-0.631024,0.001405,33.704250,-0.629621


In [91]:
from astropy.coordinates import Angle
Angle(0.000782, unit='deg').arcsec

2.8152

In [85]:
Angle(5, unit='arcsec').degree

0.001388888888888889

In [92]:
Xnn.loc[Xnn['dist']<0.00139]

,"(B, RA)","(B, DEC)",dist,ra,dec
425,17.605092,-0.391549,0.000782,17.604379,-0.391872
445,58.255379,-0.050758,0.001132,58.255258,-0.051884
454,58.237733,0.091080,0.001388,58.237029,0.089812
456,58.452225,0.069827,0.001208,58.452225,0.069827
931,58.255258,-0.051884,0.001132,58.255354,-0.050074
951,58.450958,0.071449,0.001305,58.452225,0.069827
1412,58.236925,0.089952,0.001388,58.237733,0.091080
1547,17.604379,-0.391872,0.000782,17.606217,-0.391604
2088,58.452196,0.071034,0.001208,58.452625,0.069701
